# Compute obs loc error in a single vertical column

In [13]:
import numpy as np
import xarray as xr

from xarray.core.alignment import align
from scipy import optimize

from ensemblecovariancecomputer import EnsembleCovarianceComputer
from observationoperator import PointObserver
from kalmangaincomputer import KalmanGainComputer
from errorcomputer import OptimalErrorComputer, PracticalErrorComputer

from localizationfunctions import gaspari_cohn_univariate as gaspari_cohn

In [14]:
## Where are we working
proj_dir = '/Users/zstanley/Documents/git_repos/obs_loc_for_scda'
plot_dir = proj_dir + '/plots/optimal_loc'
my_data_dir = proj_dir + '/my_data/20151206.030000'
nb_dir = proj_dir + '/notebooks'

In [15]:
ds = xr.open_dataset(my_data_dir+'/temperature_covariances_averaged_tropical_pacific.nc')
ds = ds.sel(lon=209.5, lat=12.5)

### Compute ensemble covariances

In [16]:
enscov =  EnsembleCovarianceComputer()
enscov(ds)

In [17]:
enscov

EnsembleCovarianceComputer:

    Ensemble Covariance Matrices:
        status = set

 --- 
    Ensemble Size           : 20
    Number of Trials        : 1000

### Define observation operators

In [18]:
sst = PointObserver('sst')
sst(enscov)

In [19]:
ast = PointObserver('ast')
ast(enscov)

In [20]:
sst

EnsembleCovarianceComputer:

    Name of Observation     : sst
    Which Fluid is Observed : ocn
    Which Level is Observed : 0
 --- 
    Pre-defined observation types:
        ['ast', 'sst']

### Compute error associated with 'optimal' localization strategies

In [32]:
ec = OptimalErrorComputer(enscov)
ec(sst) # can also compute for ast

In [33]:
ec

ErrorComputer:

    Number of Trials        : 1000
 --- 
    Attributes: {'len_atm': 127, 'len_ocn': 75, 'slice_atm': slice(0, 127, 1), 'slice_ocn': slice(127, 202, 1), 'num_trials': 1000, 'error_unloc_atm': 0.8757872125181404, 'error_unloc_ocn': 0.2775805515733987, 'locrad_gcr_atm': 0.16156318308125248, 'locrad_gcr_ocn': 88.41025641164275, 'error_gcr_atm': 0.060252988518646376, 'error_gcr_ocn': 0.037804065139930336, 'locrad_gcra_atm': 0.251472887384253, 'locrad_gcra_ocn': 82.80116124234733, 'locatten_gcra_atm': 0.4704406738281247, 'locatten_gcra_ocn': 1.2376699038431966, 'error_gcra_atm': 0.058880400994226464, 'error_gcra_ocn': 0.03736043506457915, 'error_eorl_atm': 0.05226892398974565, 'error_eorl_ocn': 0.03603773972321351, 'error_true_K_atm': 0.06522415105137248, 'error_true_K_ocn': 0.10304930645398098}

### Compute error associated with 'practical' localization strategies

In [27]:
pec = PracticalErrorComputer(enscov)
pec(sst, enscov)

In [28]:
pec

ErrorComputer:

    Number of Trials        : 1000
 --- 
    Attributes: {'len_atm': 127, 'len_ocn': 75, 'slice_atm': slice(0, 127, 1), 'slice_ocn': slice(127, 202, 1), 'num_trials': 1000, 'locrad_atm': 0.1, 'locrad_ocn': 130, 'error_true_K_atm': 0.06522415105137248, 'error_true_K_ocn': 0.10304930645398098, 'error_unloc_atm': 0.8757872125181404, 'error_unloc_ocn': 0.2775805515733987, 'error_practical_gcr_atm': 0.06063369909518203, 'error_practical_gcr_ocn': 0.0573194716672477, 'error_practical_cutoffloc_atm': 0.06089807990568445, 'error_practical_cutoffloc_ocn': 0.05731947166724771, 'error_truecorr_cutoffloc_atm': 0.06063369909518203, 'error_truecorr_cutoffloc_ocn': 0.0573194716672477}